<a href="https://colab.research.google.com/gist/vrtmrz/37c3efd7842e49947aaaa7f665e5020a/deploy_couchdb_to_flyio_v2_with_swap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

History:
- 18, May, 2023: Initial.
- 19, Jun., 2023: Patched for enabling swap.

In [ ]:
# Configurations
import os
os.environ['region']="nrt"
os.environ['couchUser']="alkcsa93"
os.environ['couchPwd']="c349usdfnv48fsasd"

In [ ]:
# Delete once
!rm ./fly.toml

In [ ]:
# Installation
# You have to set up your account in here.
!curl -L https://fly.io/install.sh | sh
!/root/.fly/bin/flyctl auth signup

In [ ]:
# Generate server
!/root/.fly/bin/flyctl launch  --auto-confirm --generate-name --detach --no-deploy --region ${region}
!/root/.fly/bin/fly volumes create --region ${region} couchdata --size 2 --yes

In [ ]:
# Check the toml once.
!cat fly.toml

In [ ]:
# Modify fly.toml
## Port modification
!sed -i 's/8080/5984/g'  fly.toml
## Add user into.
!echo -e "\n[env]\n  COUCHDB_USER = \"${couchUser}\"" >> ./fly.toml
## Set the location of an ini file which to save configurations persistently via erlang flags.
!echo -e "\nERL_FLAGS=\"-couch_ini /opt/couchdb/etc/default.ini /opt/couchdb/etc/default.d/ /opt/couchdb/etc/local.d /opt/couchdb/etc/local.ini /opt/couchdb/data/persistence.ini\"" >> ./fly.toml
## Mounting volumes to store data and ini file.
!echo -e "\n[mounts]\n  source=\"couchdata\"\n  destination=\"/opt/couchdb/data\"" >> ./fly.toml
!cat fly.toml

In [ ]:
# Make the Dockerfile to modify the permission of the ini file. If you want to use a specific version,  you should change `latest` here.
!echo -e "\n[build]\n  dockerfile = \"./Dockerfile\"" >> ./fly.toml

In [ ]:
!echo -e "FROM couchdb:latest\nRUN sed -i '2itouch /opt/couchdb/data/persistence.ini && chmod +w /opt/couchdb/data/persistence.ini && fallocate -l 512M /swapfile && chmod 0600 /swapfile && mkswap /swapfile && echo 10 > /proc/sys/vm/swappiness && swapon /swapfile && echo 1 > /proc/sys/vm/overcommit_memory' /docker-entrypoint.sh" > ./Dockerfile

In [ ]:
# Check dockerfile
!cat ./Dockerfile

In [ ]:
# Configure password
!/root/.fly/bin/flyctl secrets set COUCHDB_PASSWORD=${couchPwd}

In [ ]:
# Deploy server
# Be sure to shutdown after the test.
!/root/.fly/bin/flyctl deploy --detach --remote-only
!/root/.fly/bin/flyctl status

In [ ]:
import subprocess, json
result = subprocess.run(["/root/.fly/bin/flyctl","status","-j"], capture_output=True, text=True)
if result.returncode==0:
    hostname = json.loads(result.stdout)["Hostname"]
    os.environ['couchHost']="https://%s" % (hostname)
    print("Your couchDB server is https://%s/" % (hostname))
else:
    print("Something occured.")


In [ ]:
# Finish setting up the CouchDB
!curl -X POST "${couchHost}/_cluster_setup" -H "Content-Type: application/json" -d "{\"action\":\"enable_single_node\",\"username\":\"${couchUser}\",\"password\":\"${couchPwd}\",\"bind_address\":\"0.0.0.0\",\"port\":5984,\"singlenode\":true}"  --user "${couchUser}:${couchPwd}"

In [ ]:
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd/require_valid_user" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd_auth/require_valid_user" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/httpd/WWW-Authenticate" -H "Content-Type: application/json" -d '"Basic realm=\"couchdb\""' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/httpd/enable_cors" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd/enable_cors" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd/max_http_request_size" -H "Content-Type: application/json" -d '"4294967296"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/couchdb/max_document_size" -H "Content-Type: application/json" -d '"50000000"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/cors/credentials" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/cors/origins" -H "Content-Type: application/json" -d '"app://obsidian.md,capacitor://localhost,http://localhost"' --user "${couchUser}:${couchPwd}"